In [ ]:
from pyspark.sql import SparkSession

**Criando uma aplicação Spark**

In [ ]:
spark = SparkSession.builder.master("local").appName("SparkByExamples.com").getOrCreate()

**Criando um dataframe sem definir um schema**

In [ ]:
df1 = spark.createDataFrame([("Pedro", 10),("Maria", 20),("José", 40)])

In [ ]:
df1.show()

**Criando um dataframe com schema**

In [ ]:
schema = "id INT, nome STRING"
dados = [[1, "Pedro"], [2, "Maria"]]
df2 = spark.createDataFrame(dados, schema)

In [ ]:
df2.show()

**Agregando dados**

In [ ]:
from pyspark.sql.functions import sum  #função de soma (agregação)

In [ ]:
schema2 = "produtos STRING, vendas INT"
vendas = [["Caneta", 10], ["Lápis", 15], ["Caneta", 40]]
df3 = spark.createDataFrame(vendas, schema2)

In [ ]:
df3.show()

In [ ]:
# Colocando o show na mesma linha do script de agregação.
agrupado = df3.groupBy("produtos").agg(sum("vendas")).show()

**Visualizando apenas as colunas que quero**

In [ ]:
#O dataframe inteiro
df3.select("produtos","vendas").show()

In [ ]:
# Apenas a coluna vendas
df3.select("vendas").show()

In [ ]:
# Invertendo a ordem 
df3.select("vendas","produtos").show()

**Criando uma expressão (qualquer cálculo matemático)**

In [ ]:
from pyspark.sql.functions import expr

In [ ]:
# O quantidade de vendas vezes 0.2 
df3.select("produtos", "vendas", expr("vendas * 0.2")).show()

**Visualizando o schema do dataframe**

In [ ]:
df3.schema

**Visualizando as colunas do dataframe**

In [ ]:
df3.columns

**Importando um arquivo CSV e definindo um schema para esse arquivo**

In [ ]:
from pyspark.sql.types import *

In [ ]:
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
despachantes = spark.read.csv("/home/rowedder/download/despachantes.csv", header=False, schema = arqschema)

In [ ]:
despachantes.show()

In [ ]:
# Abrindo o CSV com load (um método genérico) no lugar do spark.read.csv
# Colocando o sep="," --> Para conseguir abrir o formato que o arquivo foi criado (, ou ;)
# inferSchema="True" --> Ele infere o schema automaticamente.
desp_autoschema = spark.read.load("/home/rowedder/download/despachantes.csv", header=False, format="csv", sep=",", inferSchema="True")

In [ ]:
desp_autoschema.show()

**Comparando os tipos de dados dos dataframes despachantes e desp_autoschema da coluna data**

In [ ]:
despachantes.schema

In [ ]:
desp_autoschema.schema

**Filtrando dados com WHERE**

In [ ]:
from pyspark.sql import functions as func

In [ ]:
# Filtrando vendas maior que 20.
despachantes.select("id","nome","vendas").where(func.col("vendas") > 20).show()

In [ ]:
# Filtrando vendas maiores que 20 e menores do que 40.
despachantes.select("id","nome","vendas").where((func.col("vendas") > 20)&(func.col("vendas")<40)).show()

**Renomeando colunas de um dataframe**

In [ ]:
novodf = despachantes.withColumnRenamed("nome", "nomes")
novodf.columns

In [ ]:
novodf.show()

**Alterando o tipo de dado da coluna**

In [ ]:
from pyspark.sql.functions import *

In [ ]:
#Cria-se mais uma coluna porém podemos deletá-la depois.
despachantes2 = despachantes.withColumn("data2", to_timestamp(func.col("data"),"yyyy-mm-dd"))

In [ ]:
despachantes2.printSchema()

**Filtrando apenas o ano da data completa**

In [ ]:
despachantes.select(year("data")).show()

**Filtrando as datas que aparecem apenas uma vez**

In [ ]:
despachantes.select(year("data")).distinct().show()

**Ordenando pelo nome e data**

In [ ]:
despachantes2.select("nome", year("data")).orderBy("nome").show()

**Contando a quantidade de registros por data**

In [ ]:
despachantes2.select("data").groupBy(year("data")).count().show()

**Somar a quantidade de vendas**

In [ ]:
despachantes2.select(func.sum("vendas")).show()

**Principais ações e transformações em dataframes**

**Forma tabular**

In [ ]:
despachantes.show()

**Forma de lista (take)** 

In [ ]:
despachantes.take(2)

**Collect**

In [ ]:
#Trás todos os dados na forma duma lista. Ação herdada dos RDDs.
despachantes.collect()

**Count**

In [ ]:
#Trás os numeros de linhas (row) do dataframe.
despachantes.count()

**orderBy**

In [ ]:
# Por padrão ordena do menor para o maior (crescente)
despachantes.orderBy("vendas").show()

In [ ]:
# Decrescente
despachantes.orderBy(func.col("vendas").desc()).show()

In [ ]:
# Ordenando a coluna cidade e a coluna vendas de forma decrescente.
despachantes.orderBy(func.col("cidade").desc(), func.col("vendas").desc()).show()

**Total de vendas por cidade**

In [ ]:
despachantes.groupBy("cidade").agg(sum("vendas")).show()

**Agregando as vendas por cidade e ordenando do maior para o menor**

In [ ]:
# Agreguei as vendas por cidade e ordenei do maior para o menor.
despachantes.groupBy("cidade").agg(sum("vendas")).orderBy(func.col("sum(vendas)").desc()).show()

**Filter**

In [ ]:
despachantes.show()

In [ ]:
despachantes.filter(func.col("nome") == "Deolinda Vilela").show()